# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
 # Import the zip city data. Use dtype="object" to ensure all data is read in accurately.
City_Data = pd.read_csv(
    "../WeatherPy/City_Data.csv", dtype="object", encoding="utf-8")

# Visualize
City_Data.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,0,Thompson,75,CA,1619313930,46,55.7435,-97.8558,-3.0,3.6
1,1,Khani,100,GE,1619314086,93,41.9563,42.9566,8.0,1.03
2,2,Batagay-Alyta,100,RU,1619313992,72,67.8006,130.4114,-3.2,2.1
3,3,Binga,9,ZW,1619314086,68,-17.6203,27.3414,19.31,0.57
4,4,Kavieng,15,PG,1619313991,70,-2.5744,150.7967,29.39,4.54


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
 # Configure gmaps with API key
gmaps.configure(api_key=g_key)

 # Store 'Lat' and 'Lng' into  locations 
locations = City_Data[["Latitude", "Longitude"]].astype(float)

Humidity = City_Data["Humidity"].astype(float)

In [4]:
# Create a Humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
City_Data = City_Data.dropna()
# City_Data.drop(columns=["Unnamed: 0"], index=1, inplace=True)
City_Data['Max Temp'] = City_Data['Max Temp'].astype(float)
City_Data['Wind Speed'] = City_Data['Wind Speed'].astype(float)
City_Data['Cloudiness'] = City_Data['Cloudiness'].astype(float)

# remove cities where cloudiness is not zero, max temp is not between 70 and 80 degrees f, and wind is less than 10 mph
City_Data_Clean2 = City_Data[City_Data.Cloudiness == 0]
City_Data_Clean3 = City_Data_Clean2[City_Data_Clean2['Wind Speed'] < 10]
City_Data_Clean4 = City_Data_Clean3[City_Data_Clean3['Max Temp'] < 26.6667]
City_Data_CleanFinal = City_Data_Clean4[City_Data_Clean4['Max Temp'] > 21.1111]

City_Data_CleanFinal

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
21,Caravelas,0.0,BR,1619313794,88,-17.7125,-39.2481,23.10,2.06
29,Pirajuí,0.0,BR,1619314087,56,-21.9986,-49.4572,23.00,2.57
52,Carnarvon,0.0,AU,1619313938,43,-24.8667,113.6333,24.00,3.60
53,Cuauhtémoc,0.0,MX,1619314090,7,28.4167,-106.8667,21.67,1.79
67,Buritizeiro,0.0,BR,1619314092,45,-17.3511,-44.9622,21.56,1.51
280,Blackwater,0.0,AU,1619314121,34,-23.5833,148.8833,25.22,3.36
296,Kharan,0.0,PK,1619314124,39,28.5833,65.4167,23.08,1.50
333,Florianópolis,0.0,BR,1619313843,83,-27.6146,-48.5012,23.89,2.06
417,Port Macquarie,0.0,AU,1619314135,53,-31.4333,152.9167,22.78,1.79
420,Khorramshahr,0.0,IR,1619314144,34,30.4397,48.1664,26.00,2.06


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
City_Data_CleanFinal["Hotel Name"] = " "

<ipython-input-27-119694dd2a72>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  City_Data_CleanFinal["Hotel Name"] = " "


In [22]:
# geocoordinates
target_coordinates = City_Data_CleanFinal['Latitude'], City_Data_CleanFinal['Longitude']
target_search = "Hotel"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)
response.status_code
# .json()

200

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
